In [ ]:
!pip install pdfminer.six
!pip install langchain
!pip install chromad
!pip install tiktoken
!pip install unstructured
!pip3 install pinecone-client

In [ ]:
!pip install openai

In [3]:
openAI_API_Key = 'openAPI Key here'

In [4]:
import pandas
import textwrap

In [5]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [21]:
loader = UnstructuredPDFLoader("/content/drive/MyDrive/INDICA_CHATAI/Indica_ A Deep Natural History of the Indian Subcontinent.pdf")

In [7]:
from langchain.document_loaders import UnstructuredFileLoader

In [19]:
loader = UnstructuredFileLoader("/content/Harry Potter and the Order of the Phoenix.txt")

In [ ]:
!pip install pytesseract
!pip install pdf2image

In [22]:
data = loader.load()

In [23]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 1 document(s) in your data
There are 917138 characters in your document


**Chunk your data up into smaller documents**

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)
#texts = []
#for i in range (100):
  #texts.append(text1[i])

In [25]:
print (f'Now you have {len(texts)} documents')
print(type(texts))

Now you have 1321 documents
<class 'list'>


**Create embeddings of your documents to get ready for semantic search** 

In [26]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [27]:
OPENAI_API_KEY = openAI_API_Key
PINECONE_API_KEY = 'put your pinecone api in quotes'
PINECONE_API_ENV = 'asia-northeast1-gcp'

In [28]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [29]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchain2" # put in the name of your pinecone index here


In [30]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [39]:
query = "name some crater sites in india?"
docs = docsearch.similarity_search(query, k=5)

In [40]:
print(docs)

[Document(lc_kwargs={'page_content': "of\tDelhi\tis\tthe\tRamgarh\tcrater\t(Lat:\t25°32'N,\tLong:\t76°62'E)\tin\tBaran\tdistrict\tof\n\nRajasthan\twith\tthe\ttown\tof\tRamgarh\tsituated\tat\tits\tcentre.\tAlthough\tit\tis\tstill\n\nbeing\tdebated\twhether\tthis\tis\ta\tcrater\tor\tmerely\ta\tlarge,\texposed\tvolcanic\tvent,\n\nminute\tglass\tshards\tembedded\twithin\tits\tlayers\tsuggest\tthat\tit\tis\tmore\tlikely\tto\n\nbe\tan\timpact\tcrater.\tAnother\tcandidate\tcrater\tin\tthe\tArabian\tSea\tlies\toffshore\n\nbetween\tMumbai\tand\tthe\tGulf\tof\tCambay\tand\tcan\tbe\tseen\tfrom\tspace.\tCalled\n\nthe\tShiva\tcrater,\tits\tdiscoverers\tclaim\tto\thave\tfound\ta\tcircular\tand\tunusually\n\ndeep\tdepression\ton\tan\totherwise\tflat\tseabed\tand\tbelieve\tthat\tthe\tmassive\tmeteor\n\nresponsible\tfor\tit\tfell\taround\t65\tmillion\tyears\tago,\tcreating\tenough\tdestruction\tto\n\nend\tthe\treign\tof\tthe\tdinosaurs.\tHowever,\twhether\tor\tnot\tit\tis\tindeed\ta\tmeteor\n\nimpact\t

Query the document:


In [33]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [52]:
llm = OpenAI(temperature=0.5, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")


In [53]:
def askme(query):
  docs = docsearch.similarity_search(query, k=3)
  ans = chain.run(input_documents=docs, question=query)
  print (textwrap.fill(ans, width=80))

In [54]:
query = "tell me about crater lake in maharastra?"

In [55]:
askme(query)

 The Lonar crater in Buldhana district of central Maharashtra is a popular
tourist destination. It is 2 kilometres wide and 170 metres deep, and at its
centre lies a placid blue lake dotted with temples on the periphery. It is the
only natural hypervelocity impact crater in the basaltic rock in the world.


In [56]:
query = "my friend ankit is visiting madhya pradesh what ecological wonders can he see?"

In [57]:
askme(query)

 Your friend Ankit can see the fossil sites in western and central Madhya
Pradesh that Vishal Verma has been meticulously cataloguing and saving from
being lost forever. He can also see the geological monuments and fossil sites
that many other people are conserving with no resources in the face of
administrative apathy.
